In [1]:
# Import modules/packages
import os
import numpy as np

# Import third-party modules/packages
# import utils
from utils import File, Command, NiiFile, LogFile
from typing import Optional, Union

In [31]:
class ReconMRtrix (object):
    '''Class that contains the associated wrapper functions for performing
    tractography using MRtrix v3.x.
    
    Attributes:
        *
    '''
    
    nii_file: NiiFile = ""
    bval: File = ""
    bvec: File = ""
    json_file: File = ""
    log: LogFile = ""
    
    def __init__(self,
                 nii_file: str,
                 bval: str,
                 bvec: str,
                 json_file: str = "",
                 log: str = "") -> None:
        '''Init doc string for ReconMRtrix class.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.file
            "file.nii"
        
        Args:
            nii_file: Input file path to DWI NIFTI file.
            bval: Input file path to corresponding bval file.
            bvec: Input file path to corresponding bvec file.
            json_file: Input file path to corresponding JSON (sidecar) file.
            log: Log filename for output log file (, need not exist at runtime).
        '''
        self.nii_file: str = nii_file
        self.bval: str = bval
        self.bvec: str = bvec
        
        self.nii_file = NiiFile(self.nii_file)
        self.bval = File(self.bval)
        self.bvec = File(self.bvec)
        
        if json_file:
            self.json_file: str = json_file
            self.json_file = File(self.json_file)
        else:
            self.json_file = ""
            self.json_file = File(self.json_file)
            
        if log:
            self.log: str = log
            self.log = LogFile(self.log)
        else:
            self.log: str = ""
            self.log = LogFile(self.log)
        
    class Mif(File):
        '''Creates MIF files for use with MRtrix executables. Inherits 
        methods and properties from the File class.
        
        Attributes:
            * 
        '''
        
        def __init__(self,
                     file: File,
                    ext: str = "") -> None:
            '''Init doc string for Mif class. Inherits methods and properties
            from the File class. The desired MIF file's file extension can either
            be explicity specified (either: '.mif' or '.mif.gz'), or can be inferred
            depending on the input's file extension (e.g. if the file is not gzipped,
            then the file extension will be '.mif', and vice versa).
            
            Usage example:
                >>> dwi_obj = ReconMRtrix("file.nii",
                >>>                       "file.bval",
                >>>                       "file.bvec",
                >>>                       "file.json",
                >>>                       "file.log")
                >>> dwi_obj.Mif(dwi_obj.file)
                >>> dwi_obj.Mif.file
                >>> "file.mif"
                >>>
                >>> # To change the file extension
                >>> dwi_obj.Mif(dwi_obj.file,".mif")
                >>>
                >>> # or
                >>> dwi_obj.Mif(dwi_obj.file,".mif.gz")
            
            Args:
                file: Input filename of MIF file (need not exist).
                ext: Desired file extension of MIF file, i.e. the preferred
                    extension should be either '.mif' or '.mif.gz'.
            '''
            self.file = file
            # if ext:
            #     ext_num: int = len(ext)
            #     self.ext: str = ext
            #     self.file = self.file[:-(ext_num)] + self.ext
            if '.gz' in self.file:
                self.ext: str = self.file[-(7):]
                self.file = self.file[:-(7)] + ".mif"
            else:
                self.ext: str = self.file[-(4):]
                self.file = self.file[:-(4)] + ".mif"
            File.__init__(self,self.file,self.ext)
    
    def nifti_to_mif(self,
                     force: bool = False) -> Mif:
        '''Converts DWI NIFTI file and its associated files to MIF files.
        
        Usage example:
            >>> dwi_obj = ReconMRtrix("file.nii",
            >>>                       "file.bval",
            >>>                       "file.bvec",
            >>>                       "file.json",
            >>>                       "file.log")
            >>> dwi_obj.nifti_to_mif()
            >>> Mif
        
        Args:
            force: Force overwrite of existing MIF file.
        
        Returns:
            mif_file: Mif file object.
        '''
        # [DEPRECATED] old method of getting output mif file name 
        # [path, filename, _ext] = self.nii_file.file_parts()
        # mif_file: str = os.path.join(path,filename + ".mif")
        # mif_file: Mif = self.Mif(mif_file)
        
        mif_file: str = self.nii_file.abs_path()
        mif_file: Mif = self.Mif(mif_file,".mif")
        
        mr_convert = Command("mrconvert")
        if force:
            mr_convert.cmd_list.append("-force")
        if self.json_file:
            mr_convert.cmd_list.append("-json_import")
            mr_convert.cmd_list.append(self.json_file.file)
        mr_convert.cmd_list.append("-fslgrad")
        mr_convert.cmd_list.append(f"{self.bvec.file}")
        mr_convert.cmd_list.append(f"{self.bval.file}")
        mr_convert.cmd_list.append(f"{self.nii_file.file}")
        mr_convert.cmd_list.append(f"{mif_file.file}")
        
        mr_convert.run(self.log)
        return mif_file
    
    def estimate_response(self,
                          mif: str,
                          method: str = "dhollander"):
        '''doc-string'''
        [path, filename, ext] = self.nii_file.file_parts()
        
        wm_res = os.path.join(path,filename + "_response_wm.txt")
        gm_res = os.path.join(path,filename + "_response_gm.txt")
        csf_res = os.path.join(path,filename + "_response_csf.txt")
        
        dwi_response = Command("dwi2response")
        dwi_response.cmd_list.append(method)
        dwi_response.cmd_list.append(mif)
        dwi_response.cmd_list.append(wm_res)
        dwi_response.cmd_list.append(gm_res)
        dwi_response.cmd_list.append(csf_res)
        
        dwi_response.run(self.log)
        
        return wm_res, gm_res, csf_res
    
    def mr_upsample(mif,output):
        '''doc-string'''
    
    def create_mask(mif,frac_int):
        '''doc-string'''
        
    def ss3t_csd(mif,mask,wm_res,gm_res,csf_res):
        '''doc-string'''
    
    def bias_field_correction(wm_fod,gm,csf,mask):
        '''doc-string'''
    
    def compute_dec_map(wm_fod,mask):
        '''doc-string'''
    
    def mr_trk_global(wm_fod,mask,stream_lines, cutoff):
        '''doc-string'''
    
    # write visualization functions later

In [3]:
class DTKrecon(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def dti_fit(dwi,mask,bvec,bval,threshold):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
        # DWIxfm.extract_b0s
    
    def dti_recon(dwi,mask,threshold):
        '''doc-string'''
    
    # write visualization functions later

In [4]:
class DWIxfm(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def compute_linear_xfm(dwi,template,dof):
        '''doc-string'''
    
    def compute_non_linear_xfm():
        '''doc-string'''
    
    def compute_combined_xfm():
        '''doc-string:
        computes combined linear and non-linear transform.'''
    
    def applywarp():
        '''doc-string'''

In [5]:
def test(file: File):
    pass

In [6]:
f = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"
b = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval"
e = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec"
j = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json"
log = "sub-test.log"

In [32]:
mm = ReconMRtrix(f,b,e,j,log)

In [33]:
mm.nii_file.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [34]:
mm.nii_file.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [35]:
mf = mm.Mif(mm.nii_file.file,".mif")
# mf = mm.Mif(mm.nii_file.file)
mf.ext

'.nii.gz'

In [36]:
mf.ext = ".mif"; mf.ext

'.mif'

In [37]:
mf.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [38]:
mi = mm.nifti_to_mif(force=True)

Running: mrconvert -force -json_import ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif
Running: mrconvert -force -json_import ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.json -fslgrad ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bvec ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.bval ../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C0

In [39]:
mi.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif'

In [40]:
[wm_res,gm_res,cxf_res] = mm.estimate_response(mif=mi.file)

Running: dwi2response dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_gm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_csf.txt
Running: dwi2response dhollander /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub

dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/dwi2response-tmp-0T91HA/
dwi2response: Importing DWI data (/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.mif)...
dwi2response: Changing to scratch directory (/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/dwi2response-tmp-0T91HA/)
dwi2response: Computing brain mask (dwi2mask)...
dwi2response: -------
dwi2response: 2 unique b-value(s) detected: 0,2000 with 7,61 volumes
dwi2response: -------
dwi2response: Preparation:
dwi2response: * Eroding brain mask by 3 pass(es)...
dwi2response:   [ mask: 119075 -> 80199 ]
dwi2response: * Computing signal decay metric (SDM):
dwi2response:  * b=0...
dw

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_wm.txt',
 '/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_gm.txt',
 '/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi_response_csf.txt')